# 第5章 合并
2020.04.28

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、append与assign

### 1. append方法
#### （a）利用序列添加行（必须指定name）

In [4]:
df_append = df.loc[:3,['Gender','Height']].copy() #loc方法,因此对行进行切片的末端3是包含的.
df_append

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167


In [22]:
s = pd.Series({'Gender':'F','Height':188},name='new_row')# 指定name,作为行索引

In [24]:
df_append.append(s)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_row,F,188


In [25]:
df_append # 并不会就地增加

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167


In [26]:
# 需要将原始df重新定义为append(s)之后的df
df_append=df_append.append(s)
df_append

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_row,F,188


* 如果有一系列的数据,需要作为行整合成一个df, 是使用append方法快, 还是使用df构造器快?

In [38]:
# 准备基础数据
lst=[]
for i in range(10000):
    lst.append(pd.Series(np.random.randn(10),name=str(i)))
#lst

In [28]:
pd.Series(lst[3])

0    0.675383
1   -0.033182
2    0.524223
3   -0.211350
4   -2.049467
5    0.487195
6    1.096909
7    0.071909
8    0.330070
9   -0.162457
Name: 3, dtype: float64

In [30]:
d=pd.DataFrame(lst[0]).T
d

,0,1,2,3,4,5,6,7,8,9
0,-0.616719,0.744776,0.772676,0.419898,1.166761,-0.992732,1.953261,-0.660195,0.995682,0.422189


In [32]:
def appedSeries(lst):
    d=pd.DataFrame(lst[0]).T
    for i in range(1,len(lst)):
        d=d.append(pd.Series(lst[i]))
    return(d)

In [39]:
%time d=appedSeries(lst)

Wall time: 18.8 s


In [40]:
len(d)

10000

In [41]:
#直接使用df构造器
%time d_1=pd.DataFrame(lst)

Wall time: 768 ms


In [42]:
len(d)

10000

In [43]:
del d,lst

* 使用构造器要快得多

#### （b）用DataFrame添加表

In [44]:
# append方法还可以纵向拼接两个df(需要columns一致)
df_temp = pd.DataFrame({'Gender':['F','M'],'Height':[188,176]},index=['new_1','new_2'])
df_append.append(df_temp)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_row,F,188
new_1,F,188
new_2,M,176


In [48]:
# 索引有重复也可以append--因为参数 verify_integrity 的默认值是False
df_append.append(pd.DataFrame({'Gender':['F','M'],'Height':[188,176]},index=['new_row','new_row']))#.index
#  verify_integrity : bool, default False  If True, raise ValueError on creating index with duplicates.

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_row,F,188
new_row,F,188
new_row,M,176


In [51]:
# verify_integrity=True 则会报错
df_append.append(pd.DataFrame({'Gender':['F','M'],'Height':[188,176]},index=['new_row','new_row']),verify_integrity=True)#.index
# ValueError: Indexes have overlapping values: Index(['new_row'], dtype='object')

ValueError: Indexes have overlapping values: Index(['new_row'], dtype='object')

In [52]:
df.append??

Signature: df.append(other, ignore_index=False, verify_integrity=False, sort=False) -> 'DataFrame'
Source:   
    def append(
        self, other, ignore_index=False, verify_integrity=False, sort=False
    ) -> "DataFrame":
        """
        Append rows of `other` to the end of caller, returning a new object.

        Columns in `other` that are not in the caller are added as new columns.

        Parameters
        ----------
        other : DataFrame or Series/dict-like object, or list of these
            The data to append.
        ignore_index : bool, default False
            If True, do not use the index labels.
        verify_integrity : bool, default False
            If True, raise ValueError on creating index with duplicates.
        sort : bool, default False
            Sort columns if the columns of `self` and `other` are not aligned.

            .. versionadded:: 0.23.0
            .. versionchanged:: 1.0.0

                Changed to not sort by default.

        Ret

In [56]:
# Sereis的append方法也是可以添加索引重复的数据的.
s.append(s)

Gender      F
Height    188
Gender      F
Height    188
Name: new_row, dtype: object

In [60]:
pd.DataFrame(s).T.append(s)

,Gender,Height
new_row,F,188
new_row,F,188


In [61]:
# 如果不转置
pd.DataFrame(s).append(s)
# 从Series构造DF,会默认把Series作为列构造DF
# 再加上append方法是把Series作为行添加到df里会引入缺失值, 就会引入缺失值

,new_row,Gender,Height
Gender,F,NaN,NaN
Height,188,NaN,NaN
new_row,NaN,F,188.0


### 2. assign方法
#### 该方法主要用于添加列，列名直接由参数指定：

In [62]:
s = pd.Series(list('abcd'),index=range(4))
df_append.assign(Letter=s) #增加列的同时,为列命名--和一般的有固定的形参名的函数不太一致

,Gender,Height,Letter
0,M,173,a
1,F,192,b
2,M,186,c
3,F,167,d
new_row,F,188,NaN


In [63]:
df_append.assign??
# **kwargs : dict of {str: callable or Series}

Signature: df_append.assign(**kwargs) -> 'DataFrame'
Source:   
    def assign(self, **kwargs) -> "DataFrame":
        r"""
        Assign new columns to a DataFrame.

        Returns a new object with all original columns in addition to new ones.
        Existing columns that are re-assigned will be overwritten.

        Parameters
        ----------
        **kwargs : dict of {str: callable or Series}
            The column names are keywords. If the values are
            callable, they are computed on the DataFrame and
            assigned to the new columns. The callable must not
            change input DataFrame (though pandas doesn't check it).
            If the values are not callable, (e.g. a Series, scalar, or array),
            they are simply assigned.

        Returns
        -------
        DataFrame
            A new DataFrame with the new columns in addition to
            all the existing columns.

        Notes
        -----
        Assigning multiple columns withi

#### 可以一次添加多个列：

In [64]:
df_append.assign(col1=lambda x:x['Gender']*2,
                 col2=s)

,Gender,Height,col1,col2
0,M,173,MM,a
1,F,192,FF,b
2,M,186,MM,c
3,F,167,FF,d
new_row,F,188,FF,NaN


In [76]:
# 使用类似字典的参数形式添加多列--如果传入的是list或者array,则需要长度和df的长度相等,否则会报错
df_append.assign(key1=np.random.randn(5),key2=np.random.randn(5),key3=np.random.randn(5))

,Gender,Height,key1,key2,key3
0,M,173,-0.241472,0.005398,0.727036
1,F,192,-0.085339,0.905147,-1.401578
2,M,186,-0.453074,-0.433772,-0.757614
3,F,167,-0.030031,0.930851,0.750511
new_row,F,188,0.988044,0.746956,-0.344357


In [77]:
# 但如果传入的是Series, 则会按索引对齐, 因此不需要长度相等--本质上是进行了左连接
df_append.assign(key1=pd.Series(np.random.randn(4)),key2=pd.Series(np.random.randn(5)),key3=pd.Series(np.random.randn(6)))

,Gender,Height,key1,key2,key3
0,M,173,-1.117655,-0.593386,-1.886381
1,F,192,-0.856960,0.267352,0.367387
2,M,186,0.213342,2.191576,-0.076652
3,F,167,0.603328,0.398312,0.439039
new_row,F,188,NaN,NaN,NaN


In [88]:
# 使用左连接的方式实现上述效果
df_append.join(pd.DataFrame(np.random.randn(6),columns=['key3']).assign(kye2=pd.Series(np.random.randn(5))).assign(key1=pd.Series(np.random.randn(4))))

,Gender,Height,key3,kye2,key1
0,M,173,0.455414,1.906880,0.159313
1,F,192,0.789416,-2.018561,-0.998381
2,M,186,0.306767,-1.308601,-0.702553
3,F,167,-1.542668,-1.445245,-0.694367
new_row,F,188,NaN,NaN,NaN


## 二、combine与update

### 1. comine方法
#### comine和update都是用于表的填充函数，可以根据某种规则填充

#### （a）填充对象
#### 可以看出combine方法是按照表的顺序轮流进行逐列循环的，而且自动索引对齐，缺失值为NaN，理解这一点很重要

In [121]:
df_combine_1 = df.loc[:1,['Gender','Height']].copy()
df_combine_1

,Gender,Height
0,M,173
1,F,192


In [122]:
df_combine_2 = df.loc[10:11,['Gender','Height']].copy()
df_combine_2

,Gender,Height
10,M,161
11,F,175


In [123]:
df_combine_1.combine(df_combine_2,lambda x,y:print(x,y))
# 观察打印的结果,发现确实是逐列进行合并的
# 合并前已经做了索引的补全, 类似于按索引的外连接
# 最后的结果全都是缺失值怎么理解?--lambda函数的原因

0       M
1       F
10    NaN
11    NaN
Name: Gender, dtype: object 0     NaN
1     NaN
10      M
11      F
Name: Gender, dtype: object
0     173.0
1     192.0
10      NaN
11      NaN
Name: Height, dtype: float64 0       NaN
1       NaN
10    161.0
11    175.0
Name: Height, dtype: float64


,Gender,Height
0,NaN,NaN
1,NaN,NaN
10,NaN,NaN
11,NaN,NaN


In [126]:
# 换一个函数
# func参数无默认值, 必须传入
df_combine_1.combine(df_combine_2,lambda x,y:x)

,Gender,Height
0,M,173.0
1,F,192.0
10,NaN,NaN
11,NaN,NaN


In [127]:
df_combine_1.combine(df_combine_2,lambda x,y:y)

,Gender,Height
0,NaN,NaN
1,NaN,NaN
10,M,161.0
11,F,175.0


In [166]:
# 自定义一个函数
def afunc_1(x,y):
    z=[0,1,2,3]
    return pd.Series(z,index=x.index)

In [167]:
df_combine_1.combine(df_combine_2,lambda x,y:afunc_1(x,y))

,Gender,Height
0,0,0.0
1,1,1.0
10,2,2.0
11,3,3.0


In [175]:
s1=pd.Series(['M','F',np.nan,np.nan],index=[0,1,10,11])
s2=pd.Series([np.nan,np.nan,'M','F'],index=[0,1,10,11])
s1[1],s2[11]

('F', 'F')

In [191]:
# 进一步改进自定义函数
def afunc_2(x,y):
    #z=[x[0],x[1],y[10],y[11]]
    z=[]
    print(x.index,y.index)
    for i in [0,1,10,11]:
        if x[i]!=np.nan:
            z.append(x[i])
        elif x[i]==np.nan:
            if y[i]!=np.nan:
                z.append(y[i])
            elif y[i]==np.nan:
                z.append(np.nan)
    #print(x,'\n',y,'\n',z)
    print(z)
    return pd.Series(z,index=x.index)
df_combine_1.combine(df_combine_2,lambda x,y:afunc_2(x,y))

Int64Index([0, 1, 10, 11], dtype='int64') Int64Index([0, 1, 10, 11], dtype='int64')
['M', 'F', nan, nan]
Int64Index([0, 1, 10, 11], dtype='int64') Int64Index([0, 1, 10, 11], dtype='int64')
[173.0, 192.0, nan, nan]


,Gender,Height
0,M,173.0
1,F,192.0
10,NaN,NaN
11,NaN,NaN


In [192]:
afunc_2(s1,s2)

Int64Index([0, 1, 10, 11], dtype='int64') Int64Index([0, 1, 10, 11], dtype='int64')
['M', 'F', nan, nan]


0       M
1       F
10    NaN
11    NaN
dtype: object

In [146]:
pd.Series([4,5,6],index=df_c_1.index)

0    4
1    5
2    6
dtype: int64

In [215]:
# 进一步改进自定义函数
def afunc_3(x,y):
    #z=[x[0],x[1],y[10],y[11]]
    z=[]
    print(x.index,y.index)
    for i in y.index:
        if (x[i]==np.nan)&(y[i]==np.nan):
            z.append(np.nan)
        elif x[i]!=None:                
            z.append(x[i])
        elif y[i]!=None:
            z.append(y[i])
    print(x)
    print(y)
    print(z)
    #print(z)
    return pd.Series(z,index=y.index)
d_=df_combine_1.combine(df_combine_2,lambda x,y:afunc_3(x,y))
d_
# 为什么后两行是NaN??

Int64Index([0, 1, 10, 11], dtype='int64') Int64Index([0, 1, 10, 11], dtype='int64')
0       M
1       F
10    NaN
11    NaN
Name: Gender, dtype: object
0     NaN
1     NaN
10      M
11      F
Name: Gender, dtype: object
['M', 'F', nan, nan]
Int64Index([0, 1, 10, 11], dtype='int64') Int64Index([0, 1, 10, 11], dtype='int64')
0     173.0
1     192.0
10      NaN
11      NaN
Name: Height, dtype: float64
0       NaN
1       NaN
10    161.0
11    175.0
Name: Height, dtype: float64
[173.0, 192.0, nan, nan]


,Gender,Height
0,M,173.0
1,F,192.0
10,NaN,NaN
11,NaN,NaN


In [214]:
d_.loc[10,'Gender']

nan

In [216]:
# 根据传入的函数,对两个df按索引逐行进行筛选
# 
df_combine_1.combine??
# func : function
#            Function that takes two series as inputs and return a Series or a
#            scalar. Used to merge the two dataframes column by columns.

Signature:
df_combine_1.combine(
    other: 'DataFrame',
    func,
    fill_value=None,
    overwrite=True,
) -> 'DataFrame'
Source:   
    def combine(
        self, other: "DataFrame", func, fill_value=None, overwrite=True
    ) -> "DataFrame":
        """
        Perform column-wise combine with another DataFrame.

        Combines a DataFrame with `other` DataFrame using `func`
        to element-wise combine columns. The row and column indexes of the
        resulting DataFrame will be the union of the two.

        Parameters
        ----------
        other : DataFrame
            The DataFrame to merge column-wise.
        func : function
            Function that takes two series as inputs and return a Series or a
            scalar. Used to merge the two dataframes column by columns.
        fill_value : scalar value, default None
            The value to fill NaNs with prior to passing any column to the
            merge func.
        overwrite : bool, default True
         

In [217]:
# 再看一个例子
df_c_1 = df.loc[:2,['Gender','Height']].copy().replace(186,np.nan)
df_c_1

,Gender,Height
0,M,173.0
1,F,192.0
2,M,NaN


In [218]:
df_c_2 = df.loc[2:4,['Gender','Height']].copy()#.replace(186,np.nan)
df_c_2

,Gender,Height
2,M,186
3,F,167
4,F,159


In [219]:
# 使用print,还是全都是NaN
df_c_1.combine(df_c_2,lambda x,y:print(x,y))

0      M
1      F
2      M
3    NaN
4    NaN
Name: Gender, dtype: object 0    NaN
1    NaN
2      M
3      F
4      F
Name: Gender, dtype: object
0    173.0
1    192.0
2      NaN
3      NaN
4      NaN
Name: Height, dtype: float64 0      NaN
1      NaN
2    186.0
3    167.0
4    159.0
Name: Height, dtype: float64


,Gender,Height
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [220]:
# 只显示了x
df_c_1.combine(df_c_2,lambda x,y:x)

,Gender,Height
0,M,173.0
1,F,192.0
2,M,NaN
3,NaN,NaN
4,NaN,NaN


In [221]:
# y
df_c_1.combine(df_c_2,lambda x,y:y)

,Gender,Height
0,NaN,NaN
1,NaN,NaN
2,M,186.0
3,F,167.0
4,F,159.0


In [222]:
# combine_first 函数--使用参数df来填充,如果索引不一致,则相当于合并
# 不需要传入函数
df_combine_1.combine_first(df_combine_2)

,Gender,Height
0,M,173.0
1,F,192.0
10,M,161.0
11,F,175.0


In [223]:
# 再看一个 combine_first 例子
df_1=df.loc[:1,['Gender','Height']].copy().replace('F',np.nan)
df_1

,Gender,Height
0,M,173
1,NaN,192


In [224]:
df_2=df.loc[1:3,['Gender','Height']].copy().replace('M',np.nan)
df_2

,Gender,Height
1,F,192
2,NaN,186
3,F,167


In [225]:
# df_1的缺失值对应的df_2相应位置不是缺失值,于是df_1的这个位置的缺失值就被df_2这个索引位置的值填充了.
# 索引不一致的其他行,则相当于直接append了--并不是
df_1.combine_first(df_2)

,Gender,Height
0,M,173.0
1,F,192.0
2,NaN,186.0
3,F,167.0


In [226]:
df_combine_1.combine_first??

Signature: df_combine_1.combine_first(other: 'DataFrame') -> 'DataFrame'
Source:   
    def combine_first(self, other: "DataFrame") -> "DataFrame":
        """
        Update null elements with value in the same location in `other`.

        Combine two DataFrame objects by filling null values in one DataFrame
        with non-null values from other DataFrame. The row and column indexes
        of the resulting DataFrame will be the union of the two.

        Parameters
        ----------
        other : DataFrame
            Provided DataFrame to use to fill null values.

        Returns
        -------
        DataFrame

        See Also
        --------
        DataFrame.combine : Perform series-wise operation on two DataFrames
            using a given function.

        Examples
        --------

        >>> df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
        >>> df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
        >>> df1.combine_first(df2)
             A    B
        

#### （b）一些例子
#### 例①：根据列均值的大小填充

In [227]:
# 例子1
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [8, 7], 'B': [6, 5]})
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B
0,8,6
1,7,5


In [230]:
df1.combine(df2,np.minimum)

,A,B
0,1,3
1,2,4


In [231]:
df1.combine(df2,np.maximum)

,A,B
0,8,6
1,7,5


#### 例②：索引对齐特性（默认状态下，后面的表没有的行列都会设置为NaN）

In [232]:
df2 = pd.DataFrame({'B': [8, 7], 'C': [6, 5]},index=[1,2])
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B,C
0,NaN,NaN,NaN
1,NaN,8.0,6.0
2,NaN,7.0,5.0


In [236]:
# fill_value 参数设置填充值--这种情况下,df1的A列的值也有可能被显示出来,只有df1,df2都没有值的位置才被设置成了填充值.
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=0)

,A,B,C
0,1.0,0.0,0.0
1,2.0,8.0,6.0
2,0.0,7.0,5.0


In [237]:
# 改为一个特别大的值--只有df1,df2的行列索引重叠的位置,才会按照二者之中均值较大的来显示,其他二者之一缺失的,都显示了填充值.
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=1000)
# fill_value : scalar value, default None
#            The value to fill NaNs with prior to passing any column to the merge func.

,A,B,C
0,1000.0,1000.0,1000.0
1,1000.0,8.0,1000.0
2,1000.0,7.0,1000.0


#### 例③：使得df1原来符合条件的值不会被覆盖

In [238]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,overwrite=False) 

,A,B,C
0,1.0,NaN,NaN
1,2.0,8.0,6.0
2,NaN,7.0,5.0


In [239]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=1000,overwrite=False)

,A,B,C
0,1.0,1000.0,1000.0
1,2.0,8.0,1000.0
2,NaN,7.0,1000.0


In [240]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=0,overwrite=False)

,A,B,C
0,1.0,0.0,0.0
1,2.0,8.0,6.0
2,NaN,7.0,5.0


#### 例④：在新增匹配df2的元素位置填充-1

In [241]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=-1)

,A,B,C
0,1.0,-1.0,-1.0
1,2.0,8.0,6.0
2,-1.0,7.0,5.0


In [242]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=-1,overwrite=False)

,A,B,C
0,1.0,-1.0,-1.0
1,2.0,8.0,6.0
2,NaN,7.0,5.0


#### （c）combine_first方法
#### 这个方法作用是用df2填补df1的缺失值，功能比较简单，但很多时候会比combine更常用，下面举两个例子：

In [243]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,0.0,4.0


In [244]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
df1.combine_first(df2)

,A,B,C
0,NaN,4.0,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


In [246]:
# 如果没有缺失值, 就会按行索引合并--当行索引完全一致而列索引完全不一致时,就相当于横向拼接
df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'C': [3, 3], 'D': [1, 1]})
df1.combine_first(df2)
# 注意combine_first后, 原来属于df2的列变成浮点型了.

,A,B,C,D
0,0,4,3.0,1.0
1,0,4,3.0,1.0


In [247]:
# 当行索引完全不一致,而列索引完全一致, 则会纵向拼接
df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'A': [3, 3], 'B': [1, 1]},index=[5,6])
df1.combine_first(df2)

,A,B
0,0.0,4.0
1,0.0,4.0
5,3.0,1.0
6,3.0,1.0


In [248]:
# 当横纵索引都不一致, 能对应到df1,df2的位置,使用各自的值
# 对应不到的位置, 使用缺失值
df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'C': [3, 3], 'D': [1, 1]},index=[5,6])
df1.combine_first(df2)

,A,B,C,D
0,0.0,4.0,NaN,NaN
1,0.0,4.0,NaN,NaN
5,NaN,NaN,3.0,1.0
6,NaN,NaN,3.0,1.0


In [249]:
# 当横纵索引部分一致---行列索引完全一致的位置会保留df1的值
# df1存在但df2不存在的位置,会使用df1的值,,df2存在但df1不存在的位置,会使用df2的值
# 都不存在的位置,引入缺失值
df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'B': [3, 3], 'D': [1, 1]},index=[1,2])
df1.combine_first(df2)
# 这个是最常用的场景 ---- 用参数df去填充调用df的缺失值, 非缺失值的话, 也并不一定是"谁在前,优先使用谁的值".

,A,B,D
0,0.0,4.0,NaN
1,0.0,4.0,1.0
2,NaN,3.0,1.0


In [258]:
# 完全重合
df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]})
df1.combine_first(df2)

,A,B,C
0,0,4,1.0
1,0,4,1.0


In [256]:
df1.combine_first??

Signature: df1.combine_first(other: 'DataFrame') -> 'DataFrame'
Source:   
    def combine_first(self, other: "DataFrame") -> "DataFrame":
        """
        Update null elements with value in the same location in `other`.

        Combine two DataFrame objects by filling null values in one DataFrame
        with non-null values from other DataFrame. The row and column indexes
        of the resulting DataFrame will be the union of the two.

        Parameters
        ----------
        other : DataFrame
            Provided DataFrame to use to fill null values.

        Returns
        -------
        DataFrame

        See Also
        --------
        DataFrame.combine : Perform series-wise operation on two DataFrames
            using a given function.

        Examples
        --------

        >>> df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
        >>> df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
        >>> df1.combine_first(df2)
             A    B
        0  1.0  3

### 2. update方法

#### （a）三个特点
#### ①返回的框索引只会与被调用框的一致（默认使用左连接，下一节会介绍）
#### ②第二个框中的nan元素不会起作用
#### ③没有返回值，直接在df上操作

#### （b）例子
#### 例①：索引完全对齐情况下的操作

In [262]:
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df1.update(df2)
df1
# update 优先使用传入的参数df

,A,B
0,1,4
1,2,5
2,3,6


In [265]:
# 等价于 combine_first 之后再reindex,切去只有df2有的列--还需要做类型转换
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df2.combine_first(df1).reindex(columns=['A','B']) # 不reindex,会发现原来只属于df2的列变成了浮点型

,A,B
0,1.0,4
1,2.0,5
2,3.0,6


#### 例②：部分填充

In [266]:
df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'B': ['d', 'e']}, index=[1,2])
df1.update(df2)
df1
# 对于重复位置, 用后者的值去更新前者--使用中需注意究竟是哪种使用场景.

,A,B
0,a,x
1,b,d
2,c,e


In [267]:
# 完全重合--只保留参数df
df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'A': ['A', 'B', 'C'],
                    'B': ['X', 'Y', 'Z']})
df1.update(df2)
df1

,A,B
0,A,X
1,B,Y
2,C,Z


In [268]:
# 完全不重合--df1不会有任何改变
df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'C': ['A', 'B', 'C'],
                    'D': ['X', 'Y', 'Z']})
df1.update(df2)
df1

,A,B
0,a,x
1,b,y
2,c,z


In [269]:
# 部分重合--重合的部分用参数df
df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'B': ['A', 'B', 'C'],
                    'C': ['X', 'Y', 'Z']})
df1.update(df2)
df1

,A,B
0,a,A
1,b,B
2,c,C


#### 例③：不会使用传入df的缺失值进行填充

In [270]:
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, np.nan, 6]})
df1.update(df2)
df1

,A,B
0,1,4.0
1,2,500.0
2,3,6.0


## 三、concat方法
#### concat方法可以在两个维度上拼接，默认纵向凭借（axis=0），拼接方式默认外连接
#### 所谓外连接，就是取拼接方向的并集，而'inner'时取拼接方向（若使用默认的纵向拼接，则为列的交集）的交集
#### 下面举一些例子说明其参数：

In [271]:
df1 = pd.DataFrame({'A': ['A0', 'A1'],
                    'B': ['B0', 'B1']},
                    index = [0,1])
df2 = pd.DataFrame({'A': ['A2', 'A3'],
                    'B': ['B2', 'B3']},
                    index = [2,3])
df3 = pd.DataFrame({'A': ['A1', 'A3'],
                    'D': ['D1', 'D3'],
                    'E': ['E1', 'E3']},
                    index = [1,3])

#### 默认状态拼接：

In [272]:
# 注意传入的是df的列表,拼接时的键就是行索引--行索引不重叠则叠加在一起
pd.concat([df1,df2])

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [273]:
# 注意传入的是df的列表--行列索引都有重叠, 但concat的结果并不会像combine那样合并相同索引的行或列
pd.concat([df1,df3])
# join='outer' 参数决定了在非拼接方向上, 按外连接来处理索引--因此列索引是之前二者的并集

,A,B,D,E
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
1,A1,NaN,D1,E1
3,A3,NaN,D3,E3


In [289]:
# 当改为横向拼接时, 行索引就是之前二者的行索引的并集--由于二者的行索引有重合,因此concat后的行索引是3个
pd.concat([df1,df3],axis=1)

,A,B,A,D,E
0,A0,B0,NaN,NaN,NaN
1,A1,B1,A1,D1,E1
3,NaN,NaN,A3,D3,E3


In [274]:
pd.concat??

Signature:
pd.concat(
    objs: Union[Iterable[Union[ForwardRef('DataFrame'), ForwardRef('Series')]], Mapping[Union[Hashable, NoneType], Union[ForwardRef('DataFrame'), ForwardRef('Series')]]],
    axis=0,
    join='outer',
    ignore_index: bool = False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity: bool = False,
    sort: bool = False,
    copy: bool = True,
) -> Union[ForwardRef('DataFrame'), ForwardRef('Series')]
Source:   
def concat(
    objs: Union[
        Iterable[FrameOrSeriesUnion], Mapping[Optional[Hashable], FrameOrSeriesUnion]
    ],
    axis=0,
    join="outer",
    ignore_index: bool = False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity: bool = False,
    sort: bool = False,
    copy: bool = True,
) -> FrameOrSeriesUnion:
    """
    Concatenate pandas objects along a particular axis with optional set logic
    along the other axes.

    Can also add a layer of hierarchical indexing on the concatenation axis,
    which may 

In [276]:
# 行列索引完全重合
d1 = pd.DataFrame({'A': ['A0', 'A1'],'B': ['B0', 'B1']}, index = [0,1])
d2 = pd.DataFrame({'A': ['A2', 'A3'],'B': ['B2', 'B3']}, index = [0,1])
pd.concat([d1,d2])
# 仍然会直接拼接结果,不会处理重叠的索引

,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


In [277]:
# 横向拼接
pd.concat([d1,d2],axis=1)

,A,B,A,B
0,A0,B0,A2,B2
1,A1,B1,A3,B3


In [278]:
# ignore_index=True 会重新设置索引--要视原始索引是否有用而定
pd.concat([d1,d2],ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [280]:
pd.concat([d1,d2]).sort_index().index

Int64Index([0, 0, 1, 1], dtype='int64')

In [287]:
# 在不确定索引是否有用的情况下, 可以在最初的设置索引时,drop参数设置为False, 这样即便索引因其他各种原因丢失, 也不会影响到数据的完整性
pd.concat([d1,d2]).set_index('A',drop= False).sort_index()#.index

,A,B
A,,
A0,A0,B0
A1,A1,B1
A2,A2,B2
A3,A3,B3


#### axis=1时沿列方向拼接：

In [288]:
pd.concat([df1,df2],axis=1)

,A,B,A,B
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,NaN,NaN,A2,B2
3,NaN,NaN,A3,B3


* 层次索引的concat比较复杂, 在需要拼接层次索引的时候, 一个思路是: 可以先把要拼接的df的索引放回到列中,拼接完了再重新设置索引.

#### join设置为内连接（由于axis=0，因此列取交集）：

In [290]:
pd.concat([df3,df1],join='inner')
# join参数默认为left

,A
1,A1
3,A3
0,A0
1,A1


In [291]:
pd.concat([df3,df1])
# join参数默认为outer

,A,D,E,B
1,A1,D1,E1,NaN
3,A3,D3,E3,NaN
0,A0,NaN,NaN,B0
1,A1,NaN,NaN,B1


#### join设置为外链接：

In [292]:
pd.concat([df3,df1],join='outer',sort=True) #sort设置列排序，默认为False

,A,B,D,E
1,A1,NaN,D1,E1
3,A3,NaN,D3,E3
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN


#### verify_integrity检查列是否唯一：

In [293]:
pd.concat([df3,df1],verify_integrity=True,sort=True)
# 报错
# ValueError: Indexes have overlapping values: Int64Index([1], dtype='int64')
# 可用于合并前的测试--也可直接比较二者的索引是否有重合

ValueError: Indexes have overlapping values: Int64Index([1], dtype='int64')

#### 同样，可以添加Series：

In [294]:
s = pd.Series(['X0', 'X1'], name='X')
pd.concat([df1,s],axis=1)

,A,B,X
0,A0,B0,X0
1,A1,B1,X1


In [296]:
# 将两个Series按列拼接成DF
s1 = pd.Series(['X0', 'X1'], name='X')
s2 = pd.Series(['Y0', 'Y1'], name='Y')
pd.concat([s1,s2],axis=1)

,X,Y
0,X0,Y0
1,X1,Y1


In [302]:
# concat和df构造器相比,将一系列Series转为df的速度比较
# 准备基础数据
lst=[]
for i in range(100000):
    lst.append(pd.Series(np.random.randn(10),name=str(i)))

# 定义concat函数
def concatSeriestodf(lst):
    return pd.concat(lst,axis=1)

In [303]:
%time d_1=concatSeriestodf(lst)
%time d_2=pd.DataFrame(lst)
# concat反而快一些

Wall time: 5 s
Wall time: 8.19 s


In [304]:
del lst,d_1,d_2

#### key参数用于对不同的数据框增加一个标号，便于索引：

In [305]:
pd.concat([df1,df2], keys=['x', 'y'])
pd.concat([df1,df2], keys=['x', 'y']).index

MultiIndex([('x', 0),
            ('x', 1),
            ('y', 2),
            ('y', 3)],
           )

In [306]:
pd.concat([df1,df2], keys=['df1', 'df2'])
# 将concat前的两个df的变量名(作为字符串)传给keys参数,可以得到一个层次索引的结果, 最上层的索引即为concat前的df的变量名

A   B
df1 0  A0  B0
    1  A1  B1
df2 2  A2  B2
    3  A3  B3

## 四、merge与join

### 1. merge函数
#### merge函数的作用是将两个pandas对象横向合并，遇到重复的索引项时会使用笛卡尔积，默认inner连接，可选left、outer、right连接
#### 所谓左连接，就是指以第一个表索引为基准，右边的表中如果不再左边的则不加入，如果在左边的就以笛卡尔积的方式加入
#### merge/join与concat的不同之处在于on参数，可以指定某一个对象为key来进行连接
#### 同样的，下面举一些例子：

In [307]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3']}) 
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
right2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3']})

#### 以key1为准则连接，如果具有相同的列，则默认suffixes=('_x','_y')：

In [308]:
pd.merge(left, right, on='key1')

,key1,key2_x,A,B,key2_y,C,D
0,K0,K0,A0,B0,K0,C0,D0
1,K0,K1,A1,B1,K0,C0,D0
2,K1,K0,A2,B2,K0,C1,D1
3,K1,K0,A2,B2,K0,C2,D2
4,K2,K1,A3,B3,K0,C3,D3


#### 以多组键连接：

In [309]:
pd.merge(left, right, on=['key1','key2']) #默认内连接

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


#### 默认使用inner连接，因为merge只能横向拼接，所以取行向上keys的交集，下面看如果使用how=outer参数
#### 注意：这里的how就是concat的join

In [310]:
pd.merge(left, right, how='outer', on=['key1','key2'])
# 外连接,因此引入了缺失值

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


#### 左连接：

In [311]:
pd.merge(left, right, how='left', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


#### 右连接：

In [313]:
pd.merge(left, right, how='right', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


In [315]:
# 按两个df的索引进行连接--指定参数 left_index=True, right_index=True
pd.merge(left, right, left_index=True, right_index=True)

,key1_x,key2_x,A,B,key1_y,key2_y,C,D
0,K0,K0,A0,B0,K0,K0,C0,D0
1,K0,K1,A1,B1,K1,K0,C1,D1
2,K1,K0,A2,B2,K1,K0,C2,D2
3,K2,K1,A3,B3,K2,K0,C3,D3


In [317]:
left_=left.set_index('key1')
left_

,key2,A,B
key1,,,
K0,K0,A0,B0
K0,K1,A1,B1
K1,K0,A2,B2
K2,K1,A3,B3


In [319]:
# 按一个df的索引和另一个df的列进行连接--指定参数 left_index=True, right_on='key1'
pd.merge(left_, right, left_index=True, right_on='key1')
# left 的索引key1有两个K0, right 的key1列有一个K0 ,因此结果会有 2*1 个K0

,key2_x,A,B,key1,key2_y,C,D
0,K0,A0,B0,K0,K0,C0,D0
0,K1,A1,B1,K0,K0,C0,D0
1,K0,A2,B2,K1,K0,C1,D1
2,K0,A2,B2,K1,K0,C2,D2
3,K1,A3,B3,K2,K0,C3,D3


In [320]:
pd.merge??

Signature:
pd.merge(
    left,
    right,
    how: str = 'inner',
    on=None,
    left_on=None,
    right_on=None,
    left_index: bool = False,
    right_index: bool = False,
    sort: bool = False,
    suffixes=('_x', '_y'),
    copy: bool = True,
    indicator: bool = False,
    validate=None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.

Parameters
----------
left : DataFrame
right : DataFrame or named Series
    Object to merge with.
how : {'left', 'right', 'outer', 'inner'}, default 'inner'
    Type of merge to be performed.

    * left: use only keys from left frame, similar to a SQL left outer join;
      preserve key order.
    * right: use only keys from right frame, similar to a SQL right outer join;
      pres

#### 如果还是对笛卡尔积不太了解，请务必理解下面这个例子，由于B的所有元素为2，因此需要6行：

In [321]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 2, 2]})
pd.merge(left, right, on='B', how='outer')
# 对于连接键(可以是列或者索引)的重复值, 进行连接时会做笛卡尔乘积---df1的该重复值个数乘以df2的该重复值的个数,等于merge后的该索引的个数

,A_x,B,A_y
0,1,2,4
1,1,2,5
2,1,2,6
3,2,2,4
4,2,2,5
5,2,2,6


In [322]:
pd.merge(left, right, on='B', how='inner')
# 使用有重复值的列进行连接, 不会报错, 但是和sql的逻辑不一样,可能会出现意料之外的问题--遇到使用有重复值的列进行连接时要谨慎

,A_x,B,A_y
0,1,2,4
1,1,2,5
2,1,2,6
3,2,2,4
4,2,2,5
5,2,2,6


#### validate检验的是到底哪一边出现了重复索引，如果是“one_to_one”则两侧索引都是唯一，如果"one_to_many"则左侧唯一

In [323]:
# 使用validate参数进行检验
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 3, 4]})
pd.merge(left, right, on='B', how='outer',validate='one_to_one') 
# 报错
# MergeError: Merge keys are not unique in left dataset; not a one-to-one merge
#
# validate的可能取值
#   * "one_to_one" or "1:1": check if merge keys are unique in both  left and right datasets.
#   * "one_to_many" or "1:m": check if merge keys are unique in left   dataset.
#   * "many_to_one" or "m:1": check if merge keys are unique in right  dataset.
#   * "many_to_many" or "m:m": allowed, but does not result in checks.
# 在编写具有更高的健壮性的生产代码时, 使用该参数进行事前检验, 并将结果写入日志是值得考虑的.

MergeError: Merge keys are not unique in left dataset; not a one-to-one merge

In [324]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 1]})
pd.merge(left, right, on='B', how='outer',validate='one_to_one')
# 满足validate='one_to_one'则不会报错

,A_x,B,A_y
0,1.0,2,4.0
1,2.0,1,NaN
2,NaN,3,5.0
3,NaN,4,6.0


In [326]:
pd.merge(left, right, on='B', how='outer',validate='one_to_many')
# one_to_many 是只检查left的连接键是否唯一,不检查right是否不唯一
pd.merge(left, right, on='B', how='outer',validate='many_to_one')
# many_to_one  是只检查right的连接键是否唯一,不检查left是否不唯一

,A_x,B,A_y
0,1.0,2,4.0
1,2.0,1,NaN
2,NaN,3,5.0
3,NaN,4,6.0


In [328]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 3, 4]})
pd.merge(left, right, on='B', how='outer',validate='many_to_one') 
# 不满足one_to_one, 但满足many_to_one, 不会报错
pd.merge(left, right, on='B', how='outer',validate='one_to_many') 
# 不满足one_to_one, 也不满足many_to_one, 会报错
# MergeError: Merge keys are not unique in left dataset; not a one-to-many merge

MergeError: Merge keys are not unique in left dataset; not a one-to-many merge

#### indicator参数指示了，合并后该行索引的来源

In [329]:
df1 = pd.DataFrame({'col1': [0, 1], 'col_left': ['a', 'b']})
df2 = pd.DataFrame({'col1': [1, 2, 2], 'col_right': [2, 2, 2]})
pd.merge(df1, df2, on='col1', how='outer', indicator=True) #indicator='indicator_column'也是可以的
# indicator : bool or str, default False
#    If True, adds a column to output DataFrame called "_merge" with  information on the source of each row.
#    If string, column with information on source of each row will be added to
#    output DataFrame, and column will be named value of string.
#    Information column is Categorical-type and takes on a value of "left_only"
#    for observations whose merge key only appears in 'left' DataFrame,
#    "right_only" for observations whose merge key only appears in 'right'
#    DataFrame, and "both" if the observation's merge key is found in both.

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


In [330]:
pd.merge(df1, df2, on='col1', how='outer', indicator='indicator') #indicator='indicator_column'也是可以的

,col1,col_left,col_right,indicator
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


### 2. join函数
#### join函数作用是将多个pandas对象横向拼接，遇到重复的索引项时会使用笛卡尔积，默认左连接，可选inner、outer、right连接

In [331]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                    index=['K0', 'K2', 'K3'])
left.join(right)
# 默认使用索引进行左连接--会在传入参数df的索引缺失的行引入缺失值

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [335]:
# 当有大量类似字典或json格式的数据要合并的时候,由于在构造器中字典的键会自动成为索引,因此默认使用索引进行合并的属性方法join 就比较适用.
# pd.DataFrame({'key1':'value1','key2':'value2','key3':'value3','key4':'value4','key5':'value5'})
pd.Series({'key1':'value1','key2':'value2','key3':'value3','key4':'value4','key5':'value5'},name='meta1').to_frame()\
.join(pd.Series({'key1':'v1','key2':'v2','key3':'v3','key4':'v4','key5':'v5'},name='meta2').to_frame())

,meta1,meta2
key1,value1,v1
key2,value2,v2
key3,value3,v3
key4,value4,v4
key5,value5,v5


#### 对于many_to_one模式下的合并，往往join更为方便
#### 同样可以指定key：

In [338]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})
right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                     index=['K0', 'K1'])
left.join(right, on='key') 
# 当left的索引不能用于连接时, 使用参数on 指定用left的哪一列进行连接.
# 但传入参数的df,只能用索引

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


In [339]:
left.join??

Signature: left.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False) -> 'DataFrame'
Source:   
    def join(
        self, other, on=None, how="left", lsuffix="", rsuffix="", sort=False
    ) -> "DataFrame":
        """
        Join columns of another DataFrame.

        Join columns with `other` DataFrame either on index or on a key
        column. Efficiently join multiple DataFrame objects by index at once by
        passing a list.

        Parameters
        ----------
        other : DataFrame, Series, or list of DataFrame
            Index should be similar to one of the columns in this one. If a
            Series is passed, its name attribute must be set, and that will be
            used as the column name in the resulting joined DataFrame.
        on : str, list of str, or array-like, optional
            Column or index level name(s) in the caller to join on the index
            in `other`, otherwise joins index-on-index. If multiple
            values give

#### 多层key：

In [340]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1']})
index = pd.MultiIndex.from_tuples([('K0', 'K0'), ('K1', 'K0'),
                                   ('K2', 'K0'), ('K2', 'K1')],names=['key1','key2'])
right = pd.DataFrame({'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']},
                     index=index)

In [341]:
left

,A,B,key1,key2
0,A0,B0,K0,K0
1,A1,B1,K0,K1
2,A2,B2,K1,K0
3,A3,B3,K2,K1


In [342]:
right

C   D
key1 key2        
K0   K0    C0  D0
K1   K0    C1  D1
K2   K0    C2  D2
     K1    C3  D3

In [343]:
# 传入参数df具有层次索引, 因此需要指定left中相匹配的两列进行连接
left.join(right, on=['key1','key2'])
# 层次索引的join有些复杂

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


In [346]:
# 层次索引的情况, 可以使用reset将层次索引放入df,然后使用merge方法合并
pd.merge(left,right.reset_index(),left_on=['key1','key2'],right_on=['key1','key2'],how='left')

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


In [352]:
# 将left设置为层次索引, 然后使用concat方法横向合并---join方式只能指定inner或者outer
pd.concat([left.set_index(['key1','key2']),right],axis=1,join='outer').reset_index()

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K2,K0,NaN,NaN,C2,D2
4,K2,K1,A3,B3,C3,D3


## 五、问题与练习

### 1. 问题
#### 【问题一】 请思考什么是append/assign/combine/update/concat/merge/join各自最适合使用的场景，并举出相应的例子。

#### 【问题二】 merge_ordered和merge_asof的作用是什么？和merge是什么关系？

#### 【问题三】 请构造一个多级索引与多级索引合并的例子，尝试使用不同的合并函数。

#### 【问题四】 上文提到了连接的笛卡尔积，那么当连接方式变化时（inner/outer/left/right），这种笛卡尔积规则会相应变化吗？请构造相应例子。

### 2. 练习
#### 【练习一】有2张公司的员工信息表，每个公司共有16名员工，共有五个公司，请解决如下问题：

In [518]:
e1=pd.read_csv('data/Employee1.csv')
print(e1.shape)
e1.head()

(40, 6)


,Company,Name,Age,Height,Weight,Salary
0,A,a1,47,188,63.7,25819
1,A,a3,39,172,55.9,21983
2,A,a4,43,158,62.5,21755
3,A,a6,42,182,76.9,17354
4,A,a7,49,171,94.6,6177


In [519]:
e2=pd.read_csv('data/Employee2.csv')
print(e2.shape)
e2.head()

(37, 6)


,Company,Name,Age,Height,Weight,Salary
0,A,a1,30,156,91.2,28133
1,A,a2,50,190,83.4,6673
2,A,a3,34,168,96.6,16503
3,A,a5,51,176,97.2,23294
4,A,a6,37,183,93.2,19256


#### (a) 每个公司有多少员工满足如下条件：既出现第一张表，又出现在第二张表。

In [520]:
len(e1),e1.Name.nunique(),len(e2),e2.Name.nunique()
# Name列具有唯一性--至少在每个表里是这样的

(40, 40, 37, 37)

In [523]:
name1=list(e1.Name)
name2=list(e2.Name)
print(len(name1),len(name2))
name1.extend(name2)

40 37


In [524]:
len(name1),len(set(name1))
# 一共只有61个不同的 Name

(77, 61)

In [525]:
# (a)
pd.merge(e1,e2,on=['Company', 'Name'],how='inner',validate='one_to_one')[['Company', 'Name']].groupby('Company').count()

,Name
Company,
A,3
B,4
C,4
D,2
E,3


#### (b) 将所有不符合(a)中条件的行筛选出来，合并为一张新表，列名与原表一致。

In [526]:
# 先选出符合(a)的行
e_a=pd.merge(e1,e2,on=['Company', 'Name'],how='inner')
print(e_a.shape)
e_a.head()
# 公司,姓名相同的行, height 等列的值不应该不一致啊?

(16, 10)


,Company,Name,Age_x,Height_x,Weight_x,Salary_x,Age_y,Height_y,Weight_y,Salary_y
0,A,a1,47,188,63.7,25819,30,156,91.2,28133
1,A,a3,39,172,55.9,21983,34,168,96.6,16503
2,A,a6,42,182,76.9,17354,37,183,93.2,19256
3,B,b1,49,169,95.9,29223,43,163,98.7,20716
4,B,b3,33,185,59.7,5126,41,170,79.3,16810


In [527]:
# Name具有唯一性,利用name找出只属于e1不属于e_a的name,和只属于e2不属于e_a的name
print([name for name in e1.Name.values if name not in e_a.Name.values])
print([name for name in e2.Name.values if name not in e_a.Name.values])

['a4', 'a7', 'a8', 'a9', 'a13', 'a15', 'a16', 'b5', 'b6', 'b9', 'b10', 'b14', 'c4', 'c5', 'c7', 'c9', 'd2', 'd6', 'd9', 'd11', 'e4', 'e7', 'e13', 'e14']
['a2', 'a5', 'a10', 'a12', 'a14', 'b2', 'b8', 'b11', 'b13', 'c1', 'c2', 'c6', 'c11', 'd1', 'd4', 'd7', 'd8', 'e2', 'e3', 'e6', 'e12']


In [528]:
# 从e1中找出只属于e1不属于e_a的行
e_b_1=e1.query("Name in ['a4', 'a7', 'a8', 'a9', 'a13', 'a15', 'a16', 'b5', 'b6', 'b9', 'b10', 'b14', 'c4', 'c5', 'c7', 'c9', 'd2', 'd6', 'd9', 'd11', 'e4', 'e7', 'e13', 'e14']")
print(e_b_1.shape)
# 从e2中找出只属于e1不属于e_a的行
e_b_2=e2.query("Name in ['a2', 'a5', 'a10', 'a12', 'a14', 'b2', 'b8', 'b11', 'b13', 'c1', 'c2', 'c6', 'c11', 'd1', 'd4', 'd7', 'd8', 'e2', 'e3', 'e6', 'e12']")
print(e_b_2.shape)
# 合并上述二者,即可得到满足条件(b)的
e_b=e_b_1.append(e_b_2)
print(e_b.shape)
e_b.head()

(24, 6)
(21, 6)
(45, 6)


,Company,Name,Age,Height,Weight,Salary
2,A,a4,43,158,62.5,21755
4,A,a7,49,171,94.6,6177
5,A,a8,51,168,89.5,3246
6,A,a9,36,186,62.8,3569
7,A,a13,58,190,75.9,21854


#### (c) 现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，例如：P公司在两张表的交集为{p1}，并集扣除交集为{p2,p3,p4}，那么如果后者集合的工资均值为1万元，且p1在表1的工资为13000元，在表2的工资为9000元，那么应该最后取9000元作为p1的工资，最后对于没有信息的员工，利用缺失值填充。

In [532]:
pd.merge(e1,e2,on=['Age','Height','Weight'],how='outer')

,Company_x,Name_x,Age,Height,Weight,Salary_x,Company_y,Name_y,Salary_y
0,A,a1,47,188,63.7,25819.0,NaN,NaN,NaN
1,A,a3,39,172,55.9,21983.0,NaN,NaN,NaN
2,A,a4,43,158,62.5,21755.0,NaN,NaN,NaN
3,A,a6,42,182,76.9,17354.0,NaN,NaN,NaN
4,A,a7,49,171,94.6,6177.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
72,NaN,NaN,44,165,91.5,NaN,E,e6,28818.0
73,NaN,NaN,45,171,90.0,NaN,E,e8,15832.0
74,NaN,NaN,50,186,76.0,NaN,E,e10,13772.0
75,NaN,NaN,30,163,66.5,NaN,E,e11,23893.0


In [474]:
# e_b即为并集扣除交集的集合--按公司分组计算e_b中每个公司的平均工资
e_b.groupby('Company')['Salary'].mean()

Company
A    12725.166667
B    15111.222222
C    15199.750000
D    18749.625000
E    19049.500000
Name: Salary, dtype: float64

In [495]:
# 需要修改e_a的工资
e_a.head()

,Company,Name,Age_x,Height_x,Weight_x,Salary_x,Age_y,Height_y,Weight_y,Salary_y
0,A,a1,47,188,63.7,25819,30,156,91.2,28133
1,A,a3,39,172,55.9,21983,34,168,96.6,16503
2,A,a6,42,182,76.9,17354,37,183,93.2,19256
3,B,b1,49,169,95.9,29223,43,163,98.7,20716
4,B,b3,33,185,59.7,5126,41,170,79.3,16810


In [480]:
# 使用join为e_a中的行按公司添加e_b中的平均工资
e_c=e_a.join(e_b.groupby('Company')['Salary'].mean(),on='Company')
e_c

,Company,Name,Age_x,Height_x,Weight_x,Salary_x,Age_y,Height_y,Weight_y,Salary_y,Salary
0,A,a1,47,188,63.7,25819,30,156,91.2,28133,12725.166667
1,A,a3,39,172,55.9,21983,34,168,96.6,16503,12725.166667
2,A,a6,42,182,76.9,17354,37,183,93.2,19256,12725.166667
3,B,b1,49,169,95.9,29223,43,163,98.7,20716,15111.222222
4,B,b3,33,185,59.7,5126,41,170,79.3,16810,15111.222222
5,B,b7,43,167,66.5,26546,47,189,51.1,17396,15111.222222
6,B,b15,50,174,72.2,14685,49,183,98.4,29268,15111.222222
7,C,c3,37,174,82.8,22429,52,180,73.4,29963,15199.750000
8,C,c10,38,162,82.8,20045,50,166,66.2,19212,15199.750000
9,C,c12,42,173,57.4,4319,32,159,96.7,12195,15199.750000


In [496]:
# 对于满足条件(a)的员工e_a, 找出e1中的工资和e2中的工资哪个离相应的e_b中该员工所在公司的平均工资更接近
S={}
for i in e_c.index:
    if e_c.Salary_x[i]-e_c.Salary[i]>=e_c.Salary_y[i]-e_c.Salary[i]:
        S[i]=e_c.Salary_y[i]
    else:
        S[i]=e_c.Salary_x[i]
print(S)

{0: 25819, 1: 16503, 2: 17354, 3: 20716, 4: 5126, 5: 17396, 6: 14685, 7: 22429, 8: 19212, 9: 4319, 10: 7952, 11: 12973, 12: 21569, 13: 15832, 14: 13772, 15: 14368}


In [497]:
# 添加符合条件的工资到e_c
e_c['S']=pd.Series(S)
e_c

,Company,Name,Age_x,Height_x,Weight_x,Salary_x,Age_y,Height_y,Weight_y,Salary_y,Salary,S
0,A,a1,47,188,63.7,25819,30,156,91.2,28133,12725.166667,25819
1,A,a3,39,172,55.9,21983,34,168,96.6,16503,12725.166667,16503
2,A,a6,42,182,76.9,17354,37,183,93.2,19256,12725.166667,17354
3,B,b1,49,169,95.9,29223,43,163,98.7,20716,15111.222222,20716
4,B,b3,33,185,59.7,5126,41,170,79.3,16810,15111.222222,5126
5,B,b7,43,167,66.5,26546,47,189,51.1,17396,15111.222222,17396
6,B,b15,50,174,72.2,14685,49,183,98.4,29268,15111.222222,14685
7,C,c3,37,174,82.8,22429,52,180,73.4,29963,15199.750000,22429
8,C,c10,38,162,82.8,20045,50,166,66.2,19212,15199.750000,19212
9,C,c12,42,173,57.4,4319,32,159,96.7,12195,15199.750000,4319


In [499]:
# 选取列,并重命名
e_=e_c[['Company','Name','Age_x','Height_x','Weight_x','S']]
e_.columns=['Company','Name','Age','Height','Weight','Salary']
e_

,Company,Name,Age,Height,Weight,Salary
0,A,a1,47,188,63.7,25819
1,A,a3,39,172,55.9,16503
2,A,a6,42,182,76.9,17354
3,B,b1,49,169,95.9,20716
4,B,b3,33,185,59.7,5126
5,B,b7,43,167,66.5,17396
6,B,b15,50,174,72.2,14685
7,C,c3,37,174,82.8,22429
8,C,c10,38,162,82.8,19212
9,C,c12,42,173,57.4,4319


In [500]:
# 跟满足(b)条件的e_b合并
e_b.append(e_)

,Company,Name,Age,Height,Weight,Salary
2,A,a4,43,158,62.5,21755
4,A,a7,49,171,94.6,6177
5,A,a8,51,168,89.5,3246
6,A,a9,36,186,62.8,3569
7,A,a13,58,190,75.9,21854
...,...,...,...,...,...,...
11,D,d5,50,173,85.7,12973
12,D,d10,50,155,70.3,21569
13,E,e8,58,174,68.1,15832
14,E,e10,57,189,69.4,13772


#### 【练习二】有2张课程的分数表（分数随机生成），但专业课（学科基础课、专业必修课、专业选修课）与其他课程混在一起，请解决如下问题：

In [405]:
c1=pd.read_csv('data/Course1.csv')
print(c1.shape)
c1.head()

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0


In [406]:
c2=pd.read_csv('data/Course2.csv')
print(c1.shape)
c2.head()

,课程名字,课程类别,学分,分数
0,高等数学（一）,学科基础课,4,99.0
1,数据科学与工程导论,学科基础课,3,NaN
2,专业英语,学科基础课,2,100.0
3,概率论,学科基础课,3,99.0
4,计算机系统,专业必修课,4,80.0


#### (a) 将两张表分别拆分为专业课与非专业课（结果为四张表）。
#### (b) 将两张专业课的分数表和两张非专业课的分数表分别合并。
#### (c) 不使用(a)中的步骤，请直接读取两张表合并后拆分。
#### (d) 专业课程中有缺失值吗，如果有的话请在完成(c)的同时，用组内（3种类型的专业课）均值填充缺失值后拆分。

In [408]:
#(a)
c1.课程类别.unique()
# 专业课 = ['专业选修课', '专业必修课', '学科基础课']

array(['思政类', '专业选修课', '专业必修课', '学科基础课', '体育类', '公共任意选修类', '英语类', '精品课程'],
      dtype=object)

In [413]:
c1a=c1.query("课程类别 in ['专业选修课', '专业必修课', '学科基础课']")
print(c1a.shape)
c1a.head()

(21, 4)


,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


In [414]:
c1b=c1.query("课程类别 not in ['专业选修课', '专业必修课', '学科基础课']")
print(c1b.shape)
c1b.head()

(12, 4)


,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0


In [415]:
c2.课程类别.unique()
# 专业课 = ['专业选修课', '专业必修课', '学科基础课']

array(['学科基础课', '专业必修课', '专业选修课', '英语类', '思政类', '文化传承类', '公共任意选修类'],
      dtype=object)

In [416]:
c2a=c2.query("课程类别 in ['学科基础课', '专业必修课', '专业选修课']")
print(c2a.shape)
c2a.head()

(25, 4)


,课程名字,课程类别,学分,分数
0,高等数学（一）,学科基础课,4,99.0
1,数据科学与工程导论,学科基础课,3,NaN
2,专业英语,学科基础课,2,100.0
3,概率论,学科基础课,3,99.0
4,计算机系统,专业必修课,4,80.0


In [417]:
c2b=c2.query("课程类别 not in ['学科基础课', '专业必修课', '专业选修课']")
print(c2b.shape)
c2b.head()

(9, 4)


,课程名字,课程类别,学分,分数
25,学术英语听说（二）,英语类,2,92.0
26,学术英语阅读,英语类,2,72.0
27,学术英语写作,英语类,2,98.0
28,美国社会与文化,英语类,2,77.0
29,马克思主义基本原理概论,思政类,3,95.0


In [423]:
#(b)
c1.columns==c2.columns

array([ True,  True,  True,  True])

In [421]:
c1a.append(c2a)#.shape

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0
7,开源软件设计与开发,专业选修课,3,81.0
9,大数据应用案例,专业必修课,3,91.0
11,电子商务,专业选修课,3,74.0
12,高等数学（二）,学科基础课,4,92.0
14,计算机视觉,专业选修课,3,NaN


In [422]:
c1b.append(c2b)#.shape

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
18,足球（初）,体育类,1,100.0
20,毛泽东思想和中国特色社会主义理论体系概论（一）,思政类,3,99.0
23,学术英语听说（一）,英语类,2,85.0
24,程序正义与经典案例解析,精品课程,2,76.0
25,环境可持续发展,公共任意选修类,2,76.0


In [425]:
#(c)
c1.append(c2).课程类别.unique()
# 专业课 = ['专业选修课', '专业必修课', '学科基础课']

array(['思政类', '专业选修课', '专业必修课', '学科基础课', '体育类', '公共任意选修类', '英语类', '精品课程',
       '文化传承类'], dtype=object)

In [432]:
# 专业课
cc1=c1.append(c2).query("课程类别 in ['专业选修课', '专业必修课', '学科基础课']")
print(cc1.shape)
cc1.head()

(46, 4)


,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


In [433]:
# 非专业课
cc2=c1.append(c2).query("课程类别 not in ['专业选修课', '专业必修课', '学科基础课']")
print(cc2.shape)
cc2.head()

(21, 4)


,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0


In [434]:
#(d)  专业课程中有缺失值吗，如果有的话请在完成(c)的同时，用组内（3种类型的专业课）均值填充缺失值后拆分。
cc1[cc1.分数.isnull()]
# 专业课分数有缺失值,分别为如下所示

,课程名字,课程类别,学分,分数
14,计算机视觉,专业选修课,3,NaN
1,数据科学与工程导论,学科基础课,3,NaN
11,数据伦理,专业必修课,1,NaN


In [447]:
len(cc1),len(cc1.课程名字.unique())
# 课程名字 具有唯一性

(46, 46)

In [448]:
# 将课程名字 设置为索引,然后按专业课类型分组,使用transform方法,用相应分组的均值填充缺失值.
cc1.set_index('课程名字').groupby('课程类别').transform(lambda x: x.fillna(x.mean())).head()

,学分,分数
课程名字,,
云计算应用与开发,3,96.0
社会计算,3,78.0
深度学习,3,75.0
人工智能导论,3,84.0
数据结构与算法,5,82.0


In [466]:
# 经过上述填充后,非数值列(除了设置成索引的)都被丢弃了--使用join方法补全其他列--需使用on指定cc1按照 课程名字 列join
cd1=cc1.join(cc1.set_index('课程名字').groupby('课程类别').transform(lambda x: x.fillna(x.mean())),on='课程名字',rsuffix='_fill')
cd2=cd1[['课程名字','课程类别','学分','分数_fill']]
cd2.columns=['课程名字','课程类别','学分','分数']
cd2 # 专业课

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.000000
2,社会计算,专业选修课,3,78.000000
3,深度学习,专业选修课,3,75.000000
4,人工智能导论,专业必修课,3,84.000000
6,数据结构与算法,学科基础课,5,82.000000
7,开源软件设计与开发,专业选修课,3,81.000000
9,大数据应用案例,专业必修课,3,91.000000
11,电子商务,专业选修课,3,74.000000
12,高等数学（二）,学科基础课,4,92.000000
14,计算机视觉,专业选修课,3,85.791667


In [462]:
# 缺失成绩的课程填充后的成绩
cd1[cc1.分数.isnull()]

,课程名字,课程类别,学分,分数,学分_fill,分数_fill
14,计算机视觉,专业选修课,3,NaN,3,85.791667
1,数据科学与工程导论,学科基础课,3,NaN,3,91.875000
11,数据伦理,专业必修课,1,NaN,1,81.090909


In [463]:
# 各类专业课的平均成绩
cc1.groupby('课程类别').mean()
# 确实是正确填充了

,学分,分数
课程类别,,
专业必修课,3.583333,81.090909
专业选修课,3.000000,85.791667
学科基础课,3.555556,91.875000
